In [1]:
!pip install kaggle



In [3]:
import os
from kaggle.api.kaggle_api_extended import KaggleApi

# Authenticate
api = KaggleApi()
api.authenticate()

# Dataset and path setup
dataset_name = 'andrewmvd/ocular-disease-recognition-odir5k'
download_path = 'data/odir5k'

# Create folder if not exists
os.makedirs(download_path, exist_ok=True)

# Check if dataset already unzipped (assume a known file/folder from the dataset)
expected_folder = os.path.join(download_path, 'ODIR-5K')  # folder inside the dataset
if not os.path.exists(expected_folder):
    print("⬇️ Downloading and unzipping dataset...")
    api.dataset_download_files(dataset_name, path=download_path, unzip=True)
    print("✅ Dataset downloaded and unzipped.")
else:
    print("✅ Dataset already exists and is unzipped.")

# Optional: list files to confirm
print("📂 Contents:")
print(os.listdir(download_path))


⬇️ Downloading and unzipping dataset...
Dataset URL: https://www.kaggle.com/datasets/andrewmvd/ocular-disease-recognition-odir5k
✅ Dataset downloaded and unzipped.
📂 Contents:
['ocular-disease-recognition-odir5k.zip']


In [4]:
import os
from kaggle.api.kaggle_api_extended import KaggleApi
import zipfile

# Authenticate
api = KaggleApi()
api.authenticate()

# Paths
dataset_name = 'andrewmvd/ocular-disease-recognition-odir5k'
download_path = 'data/odir5k'
zip_file_path = os.path.join(download_path, 'ocular-disease-recognition-odir5k.zip')
unzipped_folder_path = os.path.join(download_path, 'ODIR-5K')  # adjust if needed

# Create download directory if it doesn't exist
os.makedirs(download_path, exist_ok=True)

# Download if not already downloaded
if not os.path.exists(zip_file_path):
    print("⬇️ Downloading dataset...")
    api.dataset_download_files(dataset_name, path=download_path)
    print("✅ Download complete.")
else:
    print("✅ Dataset already downloaded.")

# Unzip if not already unzipped
if not os.path.exists(unzipped_folder_path):
    print("📦 Unzipping dataset...")
    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        zip_ref.extractall(download_path)
    print("✅ Unzipping complete.")
else:
    print("✅ Dataset already unzipped.")

# List contents to confirm
print("📂 Contents:")
print(os.listdir(download_path))


✅ Dataset already downloaded.
📦 Unzipping dataset...
✅ Unzipping complete.
📂 Contents:
['preprocessed_images', 'ocular-disease-recognition-odir5k.zip', 'ODIR-5K', 'full_df.csv']


In [5]:
import tensorflow as tf
print("TensorFlow version:", tf.__version__)



TensorFlow version: 2.16.2


In [6]:
import tensorflow as tf

device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
    print("WARNING: No GPU Found")
else:
    print('✅ Found GPU at:', device_name)


✅ Found GPU at: /device:GPU:0


2025-06-06 23:14:51.252174: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M3
2025-06-06 23:14:51.252565: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2025-06-06 23:14:51.252580: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2025-06-06 23:14:51.252713: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2025-06-06 23:14:51.252774: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [7]:
!pip install ipython-autotime
%load_ext autotime

time: 116 μs (started: 2025-06-06 23:14:55 +05:30)


In [8]:
!pip install -q imgaug


time: 512 ms (started: 2025-06-06 23:14:57 +05:30)


In [9]:
import numpy as np
from imgaug import augmenters as iaa
np.bool = bool  # Fix for imgaug numpy compatibility


time: 407 ms (started: 2025-06-06 23:14:59 +05:30)


In [10]:
!pip install numpy pandas matplotlib seaborn scikit-learn keras


time: 534 ms (started: 2025-06-06 23:15:02 +05:30)


In [11]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, roc_curve, auc
from sklearn.preprocessing import label_binarize
import os
import csv
from keras.models import load_model

time: 184 ms (started: 2025-06-06 23:15:04 +05:30)


In [12]:
dataset_path = '/kaggle/input/ocular-disease-recognition-odir5k'
number_of_classes = 5
img_height = 224
img_width = 224
batch_size = 16

time: 472 μs (started: 2025-06-06 23:15:07 +05:30)


In [13]:
import tensorflow as tf

eff = tf.keras.applications.EfficientNetB0(
    include_top=False,
    weights='imagenet',
    input_shape=(224, 224, 3),
)

2025-06-06 23:15:09.388755: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2025-06-06 23:15:09.388801: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


time: 1.18 s (started: 2025-06-06 23:15:09 +05:30)


In [14]:
# Create an output folder first (run this once)
import os
os.makedirs("outputs", exist_ok=True)

# Training stage
training_model_save_path = "outputs/ResNet101.keras"
training_history_save_path = "outputs/ResNet101.csv"
training_history_plot_save_path = "outputs/ResNet101.png"
training_confusion_matrix_save_path = "outputs/confusion_matrix.png"
training_roc_curve_save_path = "outputs/roc_curve.png"

# Fine-tuning stage
fine_tuning_model_save_path = "outputs/fine_tuning_model.keras"
fine_tuning_history_save_path = "outputs/fine_tuning_history.csv"
fine_tuning_history_plot_save_path = "outputs/fine_tuning_history.png"
fine_tuning_confusion_matrix_save_path = "outputs/fine_tuning_confusion_matrix.png"
fine_tuning_roc_curve_save_path = "outputs/fine_tuning_roc_curve.png"

# Combined/both training
both_training_history_save_path = "outputs/Both_history.csv"
both_training_confusion_matrix_save_path = "outputs/both_confusion_matrix.csv"

# Overall CSV summaries
accuracy_loss_csv_path = "outputs/AccuracyLossCurves.csv"
roc_auroc_csv_path = "outputs/ROCandAUROC.csv"
confusion_matrix_csv_path = "outputs/ConfusionMat.csv"


time: 1.97 ms (started: 2025-06-06 23:15:13 +05:30)


In [15]:
training_epochs = 40
training_learning_rate = 0.0001
training_loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False)
training_optimizer = tf.keras.optimizers.Adam(learning_rate = training_learning_rate)
training_metrics = ['accuracy']
training_callbacks = [
    tf.keras.callbacks.ModelCheckpoint(training_model_save_path, save_weights_only=False, save_best_only=True, mode='min'),
    tf.keras.callbacks.ReduceLROnPlateau(),
]

time: 12 ms (started: 2025-06-06 23:15:16 +05:30)


In [17]:
import os

# Define the correct dataset path
dataset_path = 'data/odir5k/ODIR-5K'  # or 'data/odir5k/preprocessed_images' if you're using that

print("Available folders/files in dataset_path:\n")
print(os.listdir(dataset_path))


Available folders/files in dataset_path:

['ODIR-5K']
time: 1.72 ms (started: 2025-06-06 23:20:17 +05:30)


In [ ]:
import os
import shutil
import random
import pandas as pd

# Set random seed
random.seed(42)

# Paths

dataset_path = 'data/odir5k/ODIR-5K'  # Folder that has 'Training Images' and 'Testing Images'
excel_path = 'data/odir5k/ODIR-5K/data.xlsx'  # Corrected Excel file path
target_dir = 'data/odir5k/ocular'     # Output directory to organize images

# Load Excel file
df = pd.read_excel(excel_path, sheet_name='Sheet1', engine='openpyxl')

# Define keyword-to-class mapping
keyword_class_map = {
    'normal fundus': 'Normal',
    'diabetic retinopathy': 'Diabetes',
    'proliferative retinopathy': 'Diabetes',
    'glaucoma': 'Glaucoma',
    'cataract': 'Cataract',
    'age-related': 'Age-Related Macular Degeneration',
    'macular degeneration': 'Age-Related Macular Degeneration'
}

# Final class folders to be created
target_classes = [
    'Age-Related Macular Degeneration',
    'Normal',
    'Diabetes',
    'Glaucoma',
    'Cataract',
    'Multilabel',
    'Other'
]

# Create output folders
for cls in target_classes:
    os.makedirs(os.path.join(target_dir, cls), exist_ok=True)

# Folders to search for images
image_folders = ['Training Images', 'Testing Images']

# Helper function to extract matched class(es)
def extract_class_set(text):
    if pd.isna(text):
        return set()

    text = str(text).lower().replace('，', ',')  # Normalize punctuation
    parts = [part.strip() for part in text.split(',') if part.strip()]
    
    matched_classes = set()
    for part in parts:
        for keyword, mapped_class in keyword_class_map.items():
            if keyword in part:
                matched_classes.add(mapped_class)

    if len(parts) > 1:
        matched_classes.add('_MULTILABEL_')

    return matched_classes

# Process each row in the Excel file
for _, row in df.iterrows():
    for eye_side, eye_col, diag_col in [('left', 'Left-Fundus', 'Left-Diagnostic Keywords'),
                                        ('right', 'Right-Fundus', 'Right-Diagnostic Keywords')]:
        img_name = row[eye_col]
        if pd.isna(img_name):
            continue

        # Find image path in folders
        image_path = None
        for folder in image_folders:
            test_path = os.path.join(dataset_path, folder, img_name)
            if os.path.exists(test_path):
                image_path = test_path
                break
        if not image_path:
            continue

        # Determine classification
        matched_classes = extract_class_set(row[diag_col])

        if '_MULTILABEL_' in matched_classes or len(matched_classes) > 1:
            class_folder = 'Multilabel'
        elif len(matched_classes) == 1:
            class_folder = list(matched_classes - {'_MULTILABEL_'})[0]
        else:
            class_folder = 'Other'

        # Destination path
        dest_path = os.path.join(target_dir, class_folder, f"{eye_side}_{img_name}")

        # Copy image
        if not os.path.exists(dest_path):
            shutil.copy(image_path, dest_path)

# Done
print(f"\n✅ Final dataset organized in: {target_dir}")
print("📂 Subfolders created for each class with eye-wise image separation.")
print("📂 Multilabel and unclassified images handled separately.")


FileNotFoundError: [Errno 2] No such file or directory: 'data/odir5k/data.xlsx'

time: 77.4 ms (started: 2025-06-06 23:58:49 +05:30)
